In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shared_functions_wlp_wls as shared
import scipy.integrate
import pyccl as ccl

/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/halo_table_cache.py:12: UserWarning: Some of the functionality of the HaloTableCache classrequires h5py to be installed.
  warn("Some of the functionality of the HaloTableCache class"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/user_supplied_ptcl_catalog.py:13: UserWarning: Most of the functionality of the sim_manager sub-package requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Most of the functionality of the sim_manager "
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/download_manager.py:39: UserWarning: Some of the functionality of the DownloadManager requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Some of the functionality of the DownloadManager requires h5py to be installed,\n"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/cached_halo_catalog.py:15: UserWarning: Most of the functionality of the

In [2]:
SURVEY = 'SDSS'
if (SURVEY=='SDSS'):
    import params as pa
elif (SURVEY=='LSST_DESI'):
    import params_LSST_DESI as pa
else:
    print "We don't have support for that survey yet; exiting."
    exit()

Goal: see how well we reproduce columns 5-8 of Table 1 from Zu & Mandelbaum 2015 ($f_{\rm sat}$, $<b_{\rm all}>$, $<b_{\rm sat}>$, $<b_{\rm cen}>$ for different stellar mass bins).

In [3]:
#Initialize the cosmology
OmM = pa.OmC_s + pa.OmB_s; OmB=pa.OmB_s; h0 = pa.HH0_s / 100.; sigma80=pa.sigma8_s; n_s0 = pa.n_s_s;

p = ccl.Parameters(Omega_c = OmM-OmB, Omega_b = OmB, h = h0, sigma8=sigma80, n_s=n_s0)
cosmo = ccl.Cosmology(p)

rho_crit = 3. * 10**10 * pa.mperMpc / (8. * np.pi * pa.Gnewt * pa.Msun)  # Msol h^2 / Mpc^3, for use with M in Msol / h (comoving distances)
rho_m = (OmM) * rho_crit # units of Msol h^2 / Mpc^3 (comoving distances)

In [4]:
# define stellar mass bin limits and z's of bins:
Mslow = np.asarray([8.5, 9.4, 9.8, 10.2, 10.6, 11.0, 11.2, 11.4, 12.])
z = np.asarray([0.02, 0.04, 0.05, 0.08, 0.12, 0.15, 0.17, 0.19])

# Define a halo mass range
lgMh = np.linspace(10., 15., 30)
Mh = 10**(lgMh)

In [5]:
# At each redshift get the halo mass function and halo bias:
HMF = [0]*len(z); bh = [0]*len(z)
for zi in range(0,len(z)):
    HMF[zi]= ccl.massfunction.massfunc(cosmo, Mh / h0, 1./ (1. + z[zi]), odelta=200.) / h0**3
    bh[zi] = ccl.massfunction.halo_bias(cosmo, Mh / h0, 1./(1.+z[zi]), odelta=200.)

Get $<N_{\rm sat}>$ and $<N_{\rm cen}>$ with $M_*$ threshold of each stellar mass bin edge and subtract appropriately to get it in the stellar mass bins plotted in Figure 8.

In [6]:
Nsat_thresh = [0]*len(Mslow)
Ncen_thresh = [0]*len(Mslow)
for mi in range(0,len(Mslow)):
    Nsat_thresh[mi] = shared.get_Nsat_Zu(Mh, 10**Mslow[mi], 'tot', 'SDSS')
    Ncen_thresh[mi] = shared.get_Ncen_Zu(Mh, 10**Mslow[mi], 'SDSS')
    
Nsat_bin = [0] *(len(Mslow) - 1)
Ncen_bin = [0] *(len(Mslow) - 1)
for bi in range(0, len(Mslow)-1):
    Nsat_bin[bi] = Nsat_thresh[bi] - Nsat_thresh[bi+1]
    Ncen_bin[bi] = Ncen_thresh[bi] - Ncen_thresh[bi+1]

Get $f_{\rm sat}$ for each bin.

In [8]:
# Get satelite fraction integrated over mass in each bin
Nsat_intM = np.zeros(len(z))
Ncen_intM = np.zeros(len(z))
fsat = np.zeros(len(z))
for zi in range(0,len(z)):
    Nsat_intM[zi] = scipy.integrate.simps(Nsat_bin[zi] * HMF[zi], np.log10(Mh / h0))
    Ncen_intM[zi] = scipy.integrate.simps(Ncen_bin[zi] * HMF[zi], np.log10(Mh / h0))
    fsat[zi] = Nsat_intM[zi] / (Nsat_intM[zi] + Ncen_intM[zi])

Now for each bin get the large scale bias for each population.

In [21]:
# First need to get the total number density for each population
nbar = np.zeros(len(z))
nbar_sat = np.zeros(len(z))
nbar_cen = np.zeros(len(z))
for zi in range(0,len(z)):
    nbar[zi] = scipy.integrate.simps(HMF[zi] * (Ncen_bin[zi] + Nsat_bin[zi]), np.log10(Mh / h0))
    nbar_sat[zi] = scipy.integrate.simps(HMF[zi] * Nsat_bin[zi], np.log10(Mh / h0))
    nbar_cen[zi] = scipy.integrate.simps(HMF[zi] * Ncen_bin[zi], np.log10(Mh / h0))

b_all = np.zeros(len(z))
b_sat = np.zeros(len(z))
b_cen = np.zeros(len(z))
for zi in range(0,len(z)):
    b_all[zi] = scipy.integrate.simps(HMF[zi] * bh[zi] * (Ncen_bin[zi] + Nsat_bin[zi]), np.log10(Mh/h0)) / nbar[zi]
    b_sat[zi] = scipy.integrate.simps(HMF[zi] * bh[zi] * Nsat_bin[zi], np.log10(Mh/h0)) / nbar_sat[zi]
    b_cen[zi] = scipy.integrate.simps(HMF[zi] * bh[zi] * Ncen_bin[zi], np.log10(Mh/h0)) / nbar_cen[zi]

Now output what we have calculated

In [22]:
output = np.column_stack((Mslow[0:-1], fsat, b_all, b_sat, b_cen))
print output

[[  8.5          0.41295794   1.11693896   1.63878521   0.74984337]
 [  9.4          0.40620852   1.14988214   1.67057363   0.7936808 ]
 [  9.8          0.40701527   1.19681983   1.71019859   0.84444483]
 [ 10.2          0.36112081   1.26693256   1.82631883   0.95074443]
 [ 10.6          0.25495285   1.4249572    2.11390202   1.1892024 ]
 [ 11.           0.15264859   1.79879035   2.70559263   1.63543181]
 [ 11.2          0.09547071   2.22272771   3.29722221   2.10931761]
 [ 11.4          0.04459227   2.87725644   4.05754522   2.82216818]]


Okay so that's all basically correct, up to small factors, which I think are probably explainable by me guessing at redshifts for each bin. That's also not too surprising, given these quantities are largely related to large scales and large scales are not the problem.